In [47]:
import os
from glob import glob
import warnings
import random
warnings.filterwarnings('ignore')

import import_ipynb
import abuse_detecting_preprocessing as pp
import abuse_detecting_databuild as b

import pandas as pd
import numpy as np

import fasttext

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization, GRU
from tensorflow.keras.layers import LeakyReLU, Input, Bidirectional, LSTM, GlobalMaxPooling1D, concatenate, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import callbacks
from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns


In [48]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
SEED = 30
seed_everything(SEED)

In [49]:
def print_score(label, pred):
    print('Precision: {:.5f}'.format(precision_score(label, pred)))
    print('Recall: {:.5f}'.format(recall_score(label, pred)))
    print('F1 Score: {:.5f}'.format(f1_score(label, pred)))
    print('Accuracy : {:.5f}'.format(np.mean(label == pred)))
    print('ROC AUC Score: {:.5f}\n'.format(roc_auc_score(label, pred)))
    

In [50]:

def _1DCNN(dropout_rate, input_shape):
    
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size = 6, input_shape = input_shape))
    model.add(LeakyReLU(alpha=0.2))
    # input_shape = (25, 100)
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size = 3))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=256, kernel_size = 1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(dropout_rate))  
    
    model.add(Dense(128))
    model.add(ReLU())
    model.add(Dense(1, activation ='sigmoid'))
    
    model.compile(loss = 'binary_crossentropy',
                  optimizer = 'Adam',
                  metrics = ['accuracy'])
    
    return model


In [51]:

def combined_pooling_lstm_model(dropout_rate, input_shape):
    
    inputs = Input(shape = input_shape)
    
    # Bidirectional LSTM layer
    x = Bidirectional(LSTM(64, return_sequences = True))(inputs)
    x = Bidirectional(LSTM(64, return_sequences = True))(x)
    
    # Average pool과 Max pool 생성
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    # 두 pool을 합친다.
    combined_pool = concatenate([avg_pool, max_pool])
    combined_pool = BatchNormalization()(combined_pool)
    
    # Dropout
    combined_pool = Dropout(dropout_rate)(combined_pool)
    
    # Output
    outputs = Dense(128)(combined_pool)
    outputs = ReLU()(outputs)
    
    outputs = Dense(1, activation='sigmoid')(outputs)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(loss = "binary_crossentropy",
                  optimizer = 'Adam',
                  metrics = ['accuracy'])
        
    return model


In [52]:

def GRU_model(dropout_rate, input_shape):
    
    model = Sequential()

    model.add(GRU(128, return_sequences = True, input_shape = input_shape))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(GRU(64, return_sequences = True))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(GRU(32, return_sequences = False))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    # output
    model.add(Dense(32))
    model.add(ReLU())
    model.add(Dense(1, activation = 'sigmoid'))
    
    # Model compile
    model.compile(loss = 'binary_crossentropy',
                  optimizer ='Adam',
                  metrics = ['accuracy'])
    
    return model


In [53]:
def build_train(routes, input_,):
    fasttext_model = fasttext.load_model('/Users/kim-yongjun/Documents/abuse_detecting/fasttext_model.bin')
    
    df = pd.DataFrame()
    for r in routes:
        d = pd.read_csv(r)
        df = pd.concat([df,d], ignore_index = True)

    N = 5
    # df = df[['문장','악플/욕설']]
    # df = df.rename(columns = {'문장' : 'message', '악플/욕설' : 'label'})
    x_train = b.making_x_train(df['message'], input_, fasttext_model, N)
    y_train = np.array(df['label'])
    
    y_train = y_train.astype(np.float32)
    y_train = y_train.reshape(-1,1)
    
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, stratify = y_train, random_state = SEED,
                                                       train_size = 0.8, test_size = 0.2)
    print('x_train :', x_train.shape, 'y_train :', y_train.shape)
    print('x_test :', x_test.shape, 'y_test :', y_test.shape)

    return x_train, y_train, x_test, y_test

In [54]:
def train(model, train_set, k, epochs):
    x_train, y_train = train_set
    # train
    kf = KFold(n_splits = k, shuffle=True)

    for idx, (train_index, val_index) in enumerate(kf.split(x_train)):
        print(f'\n--{idx + 1}/{k} fold--\n')
        
        x_train_fold, x_val_fold = x_train[train_index], x_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
        
        model.fit(x_train_fold, y_train_fold, epochs = epochs,
                  validation_data = (x_val_fold, y_val_fold), callbacks = [early_stopping])
        
    print(f'\n--{model}--\n')
    y_pred = np.round(model.predict(x_test))
    print_score(y_pred, y_test)
    
    return model

In [55]:
def my_GridSearch(model, train_set, param_grid, cv, input_shape, verbose = 0, n_jobs = 5):
    train, y = train_set

    classifier = KerasClassifier(build_fn = model, input_shape = input_shape, dropout_rate = 0.0)
    # GridSearchCV 모델로 초기화
    grid_model = GridSearchCV(estimator = classifier,
                           param_grid = param_grid,
                           scoring = 'roc_auc',
                           cv = cv)
    
    # 모델 fitting
    grid_model.fit(train, y)
    
    # 최적의 하이퍼파라미터 및 점수 출력
    print("Best parameters found: ", grid_model.best_params_)
    print("Best ROC AUC score: ", grid_model.best_score_)
    
    # 결과값 저장
    results = pd.DataFrame(grid_model.cv_results_)
    results = results.sort_values(by='mean_test_score', ascending=False)
    
    return results


In [56]:
def soft_ensemble(models, test_set):
    x_test, y_test = test_set
    pred = np.column_stack([m.predict(x_test) for m in models])
    ensembled_pred = np.round(np.mean(pred, axis = 1))

    for p in pred.transpose():
        print_score(y_test, np.round(p))
    print('--soft_ensembeld_model--')
    print_score(y_test, ensembled_pred)
    

In [57]:
def hard_ensemble(models, test_set):
    x_test, y_test = test_set
    l = len(models)
    pred = np.column_stack([np.round(m.predict(x_test)) for m in models])
    ensembled_pred = np.array([1 if np.sum(p) >= 2 else 0 for p in pred])
    # for p in pred.transpose():
    #     print_score(y_test, np.round(p))
    print('--hard_ensembeld_model--')
    print_score(y_test, ensembled_pred)


In [58]:
K = 5
_input = 40 # 단어 길이
input_shape = (_input, 100) # fasttext 모델이 100개의 벡터로 반환함.
dropout_rate = 0.0
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5) 

data_routes = glob('chatting_datas/labeled_datas/*.csv')
# data_routes = glob('/Users/kim-yongjun/Documents/chatting_data/smile_gate_abuse_data/*')
x_train, y_train, x_test, y_test = build_train(data_routes, _input)
train_set = (x_train, y_train)
test_set = (x_test, y_test)

x_train : (12154, 40, 100) y_train : (12154, 1)
x_test : (3039, 40, 100) y_test : (3039, 1)


In [59]:
cv = 5 # grid_search의 cv

param_grid = {'epochs': [6, 8, 10],
              'dropout_rate': [0.2, 0.3, 0.4]
             }
# 총 90 + 120 + 150 = 360회 수행.

In [ ]:
_1DCNN_grid = my_GridSearch(_1DCNN, train_set, param_grid, cv, input_shape)


In [65]:
_1DCNN_grid[['rank_test_score', 'params', 'mean_test_score']]

,rank_test_score,params,mean_test_score
7,1,"{'dropout_rate': 0.4, 'epochs': 8}",0.833654
6,2,"{'dropout_rate': 0.4, 'epochs': 6}",0.822915
0,3,"{'dropout_rate': 0.2, 'epochs': 6}",0.804435
8,4,"{'dropout_rate': 0.4, 'epochs': 10}",0.801163
3,5,"{'dropout_rate': 0.3, 'epochs': 6}",0.797306
5,6,"{'dropout_rate': 0.3, 'epochs': 10}",0.793532
2,7,"{'dropout_rate': 0.2, 'epochs': 10}",0.791775
4,8,"{'dropout_rate': 0.3, 'epochs': 8}",0.788748
1,9,"{'dropout_rate': 0.2, 'epochs': 8}",0.782619


In [ ]:
GRU_grid = my_GridSearch(GRU_model, train_set, param_grid, cv, input_shape)


In [14]:
GRU_grid[['rank_test_score', 'params', 'mean_test_score']]

,rank_test_score,params,mean_test_score
8,1,"{'dropout_rate': 0.4, 'epochs': 10}",0.848118
4,2,"{'dropout_rate': 0.3, 'epochs': 8}",0.839179
5,3,"{'dropout_rate': 0.3, 'epochs': 10}",0.836711
3,4,"{'dropout_rate': 0.3, 'epochs': 6}",0.829297
0,5,"{'dropout_rate': 0.2, 'epochs': 6}",0.823926
7,6,"{'dropout_rate': 0.4, 'epochs': 8}",0.822377
1,7,"{'dropout_rate': 0.2, 'epochs': 8}",0.815840
2,8,"{'dropout_rate': 0.2, 'epochs': 10}",0.796843
6,9,"{'dropout_rate': 0.4, 'epochs': 6}",0.783146


In [ ]:
lstm_grid = my_GridSearch(combined_pooling_lstm_model, train_set, param_grid, cv, input_shape)


In [16]:
# lstm_grid[['rank_test_score', 'params']]
lstm_grid[['rank_test_score', 'params', 'mean_test_score']].reset_index(drop = True)

,rank_test_score,params,mean_test_score
0,1,"{'dropout_rate': 0.4, 'epochs': 6}",0.844376
1,2,"{'dropout_rate': 0.3, 'epochs': 10}",0.836789
2,3,"{'dropout_rate': 0.2, 'epochs': 8}",0.836439
3,4,"{'dropout_rate': 0.3, 'epochs': 6}",0.829884
4,5,"{'dropout_rate': 0.2, 'epochs': 6}",0.823307
5,6,"{'dropout_rate': 0.2, 'epochs': 10}",0.820952
6,7,"{'dropout_rate': 0.4, 'epochs': 8}",0.817756
7,8,"{'dropout_rate': 0.4, 'epochs': 10}",0.808001
8,9,"{'dropout_rate': 0.3, 'epochs': 8}",0.799347


In [60]:
# _1DCNN_best = _1DCNN_grid[_1DCNN_grid['rank_test_score'] == 1]['params'].item()
# GRU_best = GRU_grid[GRU_grid['rank_test_score'] == 1]['params'].item()
# BiLSTM_best = lstm_grid[lstm_grid['rank_test_score'] == 1]['params'].item()
_1DCNN_best = {'dropout_rate': 0.4, 'epochs': 8}
GRU_best = {'dropout_rate': 0.4, 'epochs': 10}
BiLSTM_best = {'dropout_rate': 0.4, 'epochs': 6}
_1DCNN_model = _1DCNN(_1DCNN_best['dropout_rate'], input_shape)
_GRU_model =  GRU_model(GRU_best['dropout_rate'], input_shape)
_BiLSTM_model = combined_pooling_lstm_model(BiLSTM_best['dropout_rate'], input_shape)


In [61]:
_1DCNN_model = train(_1DCNN_model, train_set, K, _1DCNN_best['epochs'])
_GRU_model = train(_GRU_model, train_set, K, GRU_best['epochs'])
_BiLSTM_model = train(_BiLSTM_model, train_set, K, BiLSTM_best['epochs'])


--1/5 fold--

Epoch 1/8
304/304 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8976 - loss: 0.3016 - val_accuracy: 0.9404 - val_loss: 0.1780
Epoch 2/8
304/304 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9385 - loss: 0.1784 - val_accuracy: 0.9436 - val_loss: 0.1642
Epoch 3/8
304/304 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9453 - loss: 0.1547 - val_accuracy: 0.9486 - val_loss: 0.1547
Epoch 4/8
304/304 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9502 - loss: 0.1446 - val_accuracy: 0.9523 - val_loss: 0.1583
Epoch 5/8
304/304 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9553 - loss: 0.1257 - val_accuracy: 0.9519 - val_loss: 0.1603
Epoch 6/8
304/304 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9584 - loss: 0.1166 - val_accuracy: 0.9519 - val_loss: 0.1704
Epoch 7/8
304/304 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9645 - loss: 0.1015 - val_accuracy: 0.9478 - val_loss: 0.2246
Epoch 8/8
304/304 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9626 - loss: 0.1022 - val_accur

In [62]:
models = [_1DCNN_model, _GRU_model, _BiLSTM_model]
soft_ensemble(models, test_set)
hard_ensemble(models, test_set)
 

95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Precision: 0.95820
Recall: 0.99207
F1 Score: 0.97484
Accuracy : 0.86687
ROC AUC Score: 0.77047

Precision: 0.97827
Recall: 0.94158
F1 Score: 0.95957
Accuracy : 0.81203
ROC AUC Score: 0.86177

Precision: 0.97165
Recall: 0.97656
F1 Score: 0.97410
Accuracy : 0.84407
ROC AUC Score: 0.83978

--soft_ensembeld_model--
Precision: 0.97084
Recall: 0.98449
F1 Score: 0.97762
Accuracy : 0.85085
ROC AUC Score: 0.83811

95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
--hard_ensembeld_model--
Precision: 0.97014
Recall: 0.98413
F1 Score: 0.97709
Accuracy : 0.85112
ROC AUC Score: 0.83417



In [17]:
_1DCNN_model.save('1DCNN_model.h5')
_GRU_model.save('GRU_model.h5')
_BiLSTM_model.save('BiLSTM_model.h5')